# Data Pipeline
For this project determining which strategies to use determines the sucess of the trading bot.<br>
In the context of this project we follow this structure in 3 steps found in these folders in `src/`<br>
`backtesting/` -> `data/` -> `models/`<br>
1. `backtesting/` - Here we try out any choice of strategies we want to try using `backtrader` to find a viable technical indicator based approach to ensure that historically our selection would turn a profit.
2. `data/` - Here we then implement those technical indicators onto a `DataFrame` from historical data on a stock of our choice. This also includes the signals to indicate that we should buy or sell, this makes the `DataFrame` contain all the features and labels necessary to train model on.
3. `models/` - Here we train the machine learning models on the modified `DataFrame`s and validate that they can then accurately predict the buy and sell signals based on technical indicators<br>
Given this pipeline we can ensure the validity of the technical indicators and allow machine learning models to train on them. 

# Backtesting
We first test the strategy on historical data using technicals of our choice using `backtrader`, once we decide what technicals indicators we would like to use we then apply them in the `process_data()` method found in `src/data/data_processing.py`. After we define the modifications to the `DataFrame` then we can start training the ML models

In [12]:
import sys
import os
import sys
sys.path.insert(0, '../src')

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from data import data_processing as dp # to get modified DataFrames with Technicals

# Get DataFrame
In `src/data/data_processing.py` we can load a `DataFrame` that contains our own technical indicators using the `get_df` method and a spcecified ticker. We can modify the strategy and technical indicators in the `process_data` method to adjust to new strategies. This method will also add the signals to the df to allow ML models to train on. 

In [ ]:
df = dp.get_df("AAPL")
df

In [ ]:
# Add signals
df['ACTION'] = df.iloc[:, 12] + df.iloc[:, 13] + df.iloc[:, 14] + df.iloc[:, 15] + df.iloc[:, 16]
df

# Construct ML model
Once we have a dataframe with the technical indicators we would like to use, we can construct the ML model. We will use scikit-learn to simplify the process. Here we will scale the data, set up the pipeline, and split the data up. However most of the signals in the `DataFrame` are just "HOLD" so we need to do a lot preprocessing in regards to filling in those gaps so the ML can have more diverse labels.

# Train ML Model
We will test the model on the training data which will be a chunk of the dataframe we got from yfinance. Then once trained we can put it another chunk for validation data and adjust parameters as needed. Finally we can run the model on the test data for final analysis. 

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
import warnings

X = df.iloc[:, [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11]] # All columns except the signal (feature)
y = df.iloc[:, -1] # Just the signal column (label)

rus = RandomUnderSampler(random_state=42) 
X_res, y_res = rus.fit_resample(X, y) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_rep)


Accuracy: 0.82

Classification Report:
               precision    recall  f1-score   support

          -2       0.00      0.00      0.00        18
          -1       0.82      0.81      0.82       629
           0       0.84      0.93      0.88      1356
           1       0.61      0.26      0.37       193
           2       0.65      0.45      0.53        38

    accuracy                           0.82      2234
   macro avg       0.59      0.49      0.52      2234
weighted avg       0.81      0.82      0.81      2234



# Export Model 
Once the model is trained we can export it using the `pickle` library or another equivalent. We can then use this in a driver where we can then feed the bot live data from the `finnhub` API and then recompute the technicals used to train the bot and allow it to decide and execute trades through the Alpaca API. 

In [66]:
import pickle
filename = "../src/models/model_v1.pkl"
pickle.dump(rf_classifier, open(filename, 'wb'))

# Test model on new data 

In [63]:
new_df = df.iloc[:, [0,1, 2, 3, 5, 6, 7, 8, 9, 10, 11, -1]]
new_df.to_csv('../src/engine/refined_df.csv', index=False)